In [6]:
#manipulate dataframes in python
import pandas as pd
import sys

#make API calls with python
import requests

# sys.path.insert(0, '/path/to/application/app/folder') used to import local python file, in this case local keys
sys.path.insert(0, '../src/')
import localvars

#allows us to store results of API call cleanly
import json

# List of Years
strYears =["2010","2013", "2014", "2015", "2016", "2017"]

intState = 47
intCounty = 125

tables = pd.read_csv("../src/data/tables.csv").to_dict(orient="row")
tablesdf = pd.DataFrame(tables)
tablesdf2 = tablesdf.set_index("table", drop = False)
for index, row in tablesdf2.iterrows():
    tablename= row['table']
    #print("value: " + tablesdf2.loc[tablename,'file'])
    # Using for loop 
    metric = pd.DataFrame() #creates a new dataframe that's empty
    for i in strYears: 

        #construct the API call we will use
        if tablename[:1]=="B":
            baseAPI = "https://api.census.gov/data/%s/acs/acs5?get=%s&for=tract:*&in=state:%i%%20county:%i&key=%s" 
        elif tablename[:1]=="D":
            baseAPI = "https://api.census.gov/data/%s/acs/acs5/profile?get=%s&for=tract:*&in=state:%i%%20county:%i&key=%s" 

        calledAPI = (baseAPI % (i, tablename, intState, intCounty, localvars.apiKey))
        
        print(calledAPI)

        #call the API and collect the response
        response = requests.get(calledAPI)

        #load the response into a JSON, ignoring the first element which is just field labels
        try:
            formattedResponse = json.loads(response.text)[1:]

            #flip the order of the response from [population, zipcode] -> [zipcode, population]
            formattedResponse = [item[::-1] for item in formattedResponse]

            #store the response in a dataframe
            strColYear = "y_" + i

            metrictemp = pd.DataFrame(columns=['id', '2', '3', strColYear], data=formattedResponse)
            del metrictemp['2']
            del metrictemp['3']
            if i == "2010":
                metric=metrictemp
            else: 
                metric = metric.merge(metrictemp, on='id', how='outer')
        except:
            print("Unexpected error:", sys.exc_info()[0])
            raise

    #save that dataframe to a CSV spreadsheet
    strFileName=row['file']
    strFile = "../src/data/ProcessedData/" + strFileName + ".csv"
    strJson ="../src/data/json/m" + strFileName[1:]+ ".json"
    metric.to_csv(strFile, index=False)
    print("Output: " + strFile)
    

https://api.census.gov/data/2010/acs/acs5?get=B01001_001E&for=tract:*&in=state:47%20county:125&key=c565c84c8194e4fac85204a34580b7ced55fa09c
https://api.census.gov/data/2013/acs/acs5?get=B01001_001E&for=tract:*&in=state:47%20county:125&key=c565c84c8194e4fac85204a34580b7ced55fa09c
https://api.census.gov/data/2014/acs/acs5?get=B01001_001E&for=tract:*&in=state:47%20county:125&key=c565c84c8194e4fac85204a34580b7ced55fa09c
https://api.census.gov/data/2015/acs/acs5?get=B01001_001E&for=tract:*&in=state:47%20county:125&key=c565c84c8194e4fac85204a34580b7ced55fa09c
https://api.census.gov/data/2016/acs/acs5?get=B01001_001E&for=tract:*&in=state:47%20county:125&key=c565c84c8194e4fac85204a34580b7ced55fa09c
https://api.census.gov/data/2017/acs/acs5?get=B01001_001E&for=tract:*&in=state:47%20county:125&key=c565c84c8194e4fac85204a34580b7ced55fa09c
Output: ../src/data/ProcessedData/r1.csv
https://api.census.gov/data/2010/acs/acs5?get=B01001_002E&for=tract:*&in=state:47%20county:125&key=c565c84c8194e4fac852

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
import csv  
import json  
  
csvFilePath = '../src/data/ProcessedData/r1.csv'
jsonFilePath = '../src/data/json/m1.json'

data = {}

with open(csvFilePath) as csvFile:
    csvReader=csv.DictReader(csvFile)
    for rows in csvReader:
        id=rows['id']
        #del rows['id']
        data[id]=rows       
        print(data[id])
    
with open(jsonFilePath,'w') as jsonFile:
    jsonFile.write(json.dumps(data,indent=4))
